In [1]:
import pandas as pd
import numpy as np
import io
import statsmodels.formula.api as smf
import scipy.stats as stats
import re # эта библиотека необходима для работы с регулярными выражениями

- \d -- любая цифра.
- \D -- любая не цифра.
- \w -- любая буква.
- \W -- любая не буква.
- \s -- любой пробельный символ, включая пробел
- \S -- все символы, кроме пробельных
- \* -- любое количество вхождений предыдущего символа(группы символов), от 0 до бесконечности(группы символов)
- \+ -- любое количество вхождений предыдущего символа(группы символов), от 1 до бесконечности(группы символов)
- ? -- 0 или 1 вхождение предыдущего символа(группы символов)
- {n} -- точное количество вхождений - n раз предыдущего символа (группы символов)
- {n,m} -- количество вхождений не менее n и не более m раз предыдущего символа
- | -- логическое "ИЛИ"
- [\,] -- применяется "ИЛИ" к набору символов, любой из которых может встретиться в тексте

# $$ P(\hat{\beta_1} - t_{cr}\cdot s_1 < \beta < \hat{\beta_1} + t_{cr} \cdot s_1) = \gamma $$

#### Решение задачи 1.1 [Листок 06](https://nbviewer.org/github/artamonoff/Econometrica/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B8/List06-ConfIntervals.html): Доверительные интервалы.


In [2]:
alpha = 0.1
t_cr = stats.t.ppf(1 - alpha/2, 526-7 )

In [3]:
date = '''totwrk                       -0.169***         
                              (0.018)          
                                               
age                           2.689*           
                              (1.469)          
                                               
south                        101.568**         
                             (41.837)          
                                               
male                         87.669**          
                             (35.104)          
                                               
smsa                         -54.748*          
                             (33.123)          
                                               
yngkid                        -13.962          
                             (50.341)          
                                               
marr                          31.211           
                             (42.233)          
                                               
Constant                    3450.913***        
                             (80.726)  '''

In [4]:
# Строку с данными можно разбить на 8 подстрок в 3 мя элементами в каждой, разделённых различными количеством пробелом.
# С этим справится метод reshape() из библиотеки numpy
np.reshape(date.strip().split(), (8,3) )

array([['totwrk', '-0.169***', '(0.018)'],
       ['age', '2.689*', '(1.469)'],
       ['south', '101.568**', '(41.837)'],
       ['male', '87.669**', '(35.104)'],
       ['smsa', '-54.748*', '(33.123)'],
       ['yngkid', '-13.962', '(50.341)'],
       ['marr', '31.211', '(42.233)'],
       ['Constant', '3450.913***', '(80.726)']], dtype='<U11')

In [5]:
df = pd.DataFrame(np.reshape(date.strip().split(), (8,3) ), columns=['Coeff','Est str','std.err. str'])
df

Coeff      Est str std.err. str
0    totwrk    -0.169***      (0.018)
1       age       2.689*      (1.469)
2     south    101.568**     (41.837)
3      male     87.669**     (35.104)
4      smsa     -54.748*     (33.123)
5    yngkid      -13.962     (50.341)
6      marr       31.211     (42.233)
7  Constant  3450.913***     (80.726)

#### ([-+]?\d+.\d+)--- шаблон, который выделяет из строки десятичную дробь
- [-+] -- знак плюс или мину
- ? -- знаков может быть один, может не быть вообще
- \d+ --- неопределенное количество цифр.
- \. --- знак точки, для разделения десятичной дроби
- () --- скобки обязательны, так как мы выделяем сразу группу элементов

In [6]:
df['Estimate'] = df['Est str'].str.extract(r'([-+]?\d+.\d+)').astype(float)
df['Std.error'] = df['std.err. str'].str.extract(r'([-+]?\d+.\d+)').astype(float)
df

Coeff      Est str std.err. str  Estimate  Std.error
0    totwrk    -0.169***      (0.018)    -0.169      0.018
1       age       2.689*      (1.469)     2.689      1.469
2     south    101.568**     (41.837)   101.568     41.837
3      male     87.669**     (35.104)    87.669     35.104
4      smsa     -54.748*     (33.123)   -54.748     33.123
5    yngkid      -13.962     (50.341)   -13.962     50.341
6      marr       31.211     (42.233)    31.211     42.233
7  Constant  3450.913***     (80.726)  3450.913     80.726

In [7]:
df['lower'] = (df['Estimate'] - df['Std.error']*t_cr).round(2)
df['upper'] = (df['Estimate'] + df['Std.error']*t_cr).round(2)
df

Coeff      Est str std.err. str  Estimate  Std.error    lower    upper
0    totwrk    -0.169***      (0.018)    -0.169      0.018    -0.20    -0.14
1       age       2.689*      (1.469)     2.689      1.469     0.27     5.11
2     south    101.568**     (41.837)   101.568     41.837    32.63   170.51
3      male     87.669**     (35.104)    87.669     35.104    29.82   145.51
4      smsa     -54.748*     (33.123)   -54.748     33.123  -109.33    -0.17
5    yngkid      -13.962     (50.341)   -13.962     50.341   -96.91    68.99
6      marr       31.211     (42.233)    31.211     42.233   -38.38   100.80
7  Constant  3450.913***     (80.726)  3450.913     80.726  3317.89  3583.93